# Early Fusion

This script takes all frames at once and passes them through a cnn.

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import time
import numpy as np

## 1. Load Data

To begin, data for a middle frames model must be loaded along with the expected regional rainfall values.

In [2]:
training_datafile = "E:/31-12-2020/prepared-data/middle_train.npy"
validation_datafile = "E:/31-12-2020/prepared-data/middle_valid.npy"

training_rainfallfile = "E:/31-12-2020/prepared-data/expected_train.npy"
validation_rainfallfile = "E:/31-12-2020/prepared-data/expected_valid.npy"

In [3]:
training_videos = np.load(training_datafile)
validation_videos = np.load(validation_datafile)

training_rainfall = np.load(training_rainfallfile)[:, 2:]
validation_rainfall = np.load(validation_rainfallfile)[:, 2:]

In [4]:
# We need them in X, Y, COLOURS
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)
training_videos = np.swapaxes(training_videos, 3, 4)

validation_videos = np.swapaxes(validation_videos, 1, 2)
validation_videos = np.swapaxes(validation_videos, 2, 3)
validation_videos = np.swapaxes(validation_videos, 3, 4)

In [5]:
# Scale the data
training_videos[:, :, :, :, 0] = (training_videos[:, :, :, :, 0] - np.min(training_videos[:, :, :, :, 0])) / (np.max(training_videos[:, :, :, :, 0]) - np.min(training_videos[:, :, :, :, 0]))
training_videos[:, :, :, :, 1] = (training_videos[:, :, :, :, 1] - np.min(training_videos[:, :, :, :, 1])) / (np.max(training_videos[:, :, :, :, 1]) - np.min(training_videos[:, :, :, :, 1]))

validation_videos[:, :, :, :, 0] = (validation_videos[:, :, :, :, 0] - np.min(validation_videos[:, :, :, :, 0])) / (np.max(validation_videos[:, :, :, :, 0]) - np.min(validation_videos[:, :, :, :, 0]))
validation_videos[:, :, :, :, 1] = (validation_videos[:, :, :, :, 1] - np.min(validation_videos[:, :, :, :, 1])) / (np.max(validation_videos[:, :, :, :, 1]) - np.min(validation_videos[:, :, :, :, 1]))

## 2. Model Definition

Next, a CNN model architecture is defined.

In [26]:
def model_generator(input_shape=(28, 2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv3D(16, (2, 2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Second layer
    model.add(Conv3D(16, (2, 2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Third layer
    model.add(Conv3D(8, (2, 2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(13))
    model.add(Activation('relu'))
    
    model.add(Dense(13))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [27]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="./models/"):
    """ Trains the given model with the given dataset. """
    model.fit(
        xdata,
        ydata,
        batch_size=2,
        validation_split=0.2,
        callbacks=[tensorboard],
        epochs=50
    )
    save_model(model, models_folder + run_name + ".mdl")

In [28]:
# Flexible parameters
learning_rate = 0.10

In [29]:
run_name = "early_fusion-three_layer-16168f2-222-lr_{}-{}".format(learning_rate, int(time.time()))
tb = TensorBoard(log_dir=".\\logs\\{}".format(run_name))

In [30]:
model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])

In [31]:
train_model(run_name, tb, model, training_videos, training_rainfall)

Train on 143 samples, validate on 36 samples
Epoch 1/50
143/143 [==============================] - 41s 284ms/sample - loss: 42395.9974 - val_loss: 9947.3825
Epoch 2/50
143/143 [==============================] - 41s 285ms/sample - loss: 8675.6516 - val_loss: 9978.6487
Epoch 3/50
143/143 [==============================] - 38s 264ms/sample - loss: 8715.3417 - val_loss: 9948.6876
Epoch 4/50
143/143 [==============================] - 40s 280ms/sample - loss: 8691.0629 - val_loss: 9925.8339
Epoch 5/50
143/143 [==============================] - 28s 199ms/sample - loss: 8620.8191 - val_loss: 10053.5566
Epoch 6/50
143/143 [==============================] - 30s 207ms/sample - loss: 8612.4617 - val_loss: 9932.8554
Epoch 7/50
143/143 [==============================] - 27s 188ms/sample - loss: 8606.9343 - val_loss: 9932.3972
Epoch 8/50
143/143 [==============================] - 27s 187ms/sample - loss: 8632.9454 - val_loss: 9937.0879
Epoch 9/50
143/143 [==============================] - 28s 194ms/s